In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../input/vehicle-dataset-from-cardekho/CAR DETAILS FROM CAR DEKHO.csv')


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#Finding Unique values from dataset for categorical data
print(df['seller_type'].unique())
print(df['transmission'].unique())
print(df['owner'].unique())
print(df['fuel'].unique())

In [ ]:
#checking missing or null values

df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
#Making final dataset
final_dataset = df[[ 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner']]


In [ ]:
final_dataset.head()


In [ ]:
#this is for adding another feature 'No of years'
final_dataset['Current_Year'] = 2020

In [ ]:
final_dataset.head()

In [ ]:
#creating another derived feature
final_dataset['No_of_years'] = final_dataset['Current_Year'] - final_dataset['year']

In [ ]:
final_dataset.head()

In [ ]:
#removing unwanted features i.e year and current_year

final_dataset.drop(['year'],axis=1,inplace=True)


In [ ]:
final_dataset.drop(['Current_Year'],axis=1,inplace=True)
final_dataset.head()

In [ ]:
final_dataset.head()

In [ ]:
#One hot encoding --dummy variable trap..
final_dataset = pd.get_dummies(final_dataset,drop_first=True)

In [ ]:
final_dataset.head()

In [ ]:
#visualzation 
sns.pairplot(final_dataset)

In [ ]:
%matplotlib inline

corrmat = final_dataset.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))

#ploting heat map

g = sns.heatmap(final_dataset[top_corr_features].corr(),annot=True,cmap="RdYlGn")


In [ ]:
final_dataset.head()

In [ ]:
X = final_dataset.iloc[:,1:]
y = final_dataset.iloc[:,0]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#feature importance 

from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
Feat_Importance = pd.Series(model.feature_importances_,index=X.columns)
Feat_Importance.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 0)

In [ ]:
X_train.shape

In [ ]:
#implementing randomforest regressor
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor()

In [ ]:
#Hyperparameters tuning using randomised search cv


# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100 , stop = 1200 , num = 12)]
# Number of features to consider at every split
max_features = ['auto' , 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5 , 30 , num =6)]

# max_depth.append(None)
# Minimum number of samples required to split a node

min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
from sklearn.model_selection import RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:

rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions = rf_random.predict(X_test)

In [ ]:
sns.distplot(predictions - y_test)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:

import pickle
# open a file, where you ant to store the data
file = open('random_forest_regression_model.pkl', 'wb')

pickle.dump(rf_random,file)
